In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
def get_numpy_data(data, features, output):
    data['constant'] = 1 # add a constant column to a dataframe
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of dataframe given by the ‘features’ list into the SFrame ‘features_sframe’

    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = data[features].as_matrix(columns=None)
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’

    # this will convert the SArray into a numpy array:
    output_array = data[output].as_matrix(columns=None) 
    return(features_matrix, output_array)


If the features matrix (including a column of 1s for the constant) is stored as a 2D array (or matrix) and the regression weights are stored as a 1D array then the predicted output is just the dot product between the features matrix and the weights (with the weights on the right). Write a function ‘predict_output’ which accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and returns a 1D array ‘predictions’. e.g. in python:

In [4]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.



To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Write a short function called ‘normalize_features(feature_matrix)’, which normalizes columns of a given feature matrix. The function should return a pair ‘(normalized_features, norms)’, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.

from math import sqrt
def normalize_features(features):
    norms = np.sqrt(np.sum(features**2,axis=0))  
    normalized_features = features/norms 
        
    return (normalized_features, norms)

Normalized Features Test

In [5]:
def normalize_features(features):
    norms = np.sqrt(np.sum(features**2,axis=0))
    normlized_features = features/norms
    return (normlized_features, norms)

In [6]:
normalize_features(np.array([[3,6,9],[4,8,12]]))

(array([[ 0.6,  0.6,  0.6],
        [ 0.8,  0.8,  0.8]]), array([  5.,  10.,  15.]))

In [7]:
vec = np.sqrt(np.sum((np.array([[3,6,9],[4,8,12]]))**2,axis=0))
vec

array([  5.,  10.,  15.])

In [8]:
data = np.array([[3,6,9],[4,8,12]])
data

array([[ 3,  6,  9],
       [ 4,  8, 12]])

Review of Coordinate Descent
We seek to obtain a sparse set of weights by minimizing the LASSO cost function
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.
Pick a coordinate i Compute w[i] that minimizes the LASSO cost function Repeat the two steps for all coordinates, multiple times

 For this assignment, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:



1
2
3
       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
w[i] = ├ 0                     if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)    if ro[i] > lambda/2


where
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].

w[0] = ro[i]

9. Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
Set initial weights to [1,4,1].
Make predictions with feature matrix and initial weights.
Compute values of ro[i], where

In [9]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
train = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)

In [10]:
features = ['sqft_living','bedrooms']
output = 'price'

In [11]:
features_matrix, output_array = get_numpy_data(sales, features, output)

In [12]:
simple_features_matrix,norms = normalize_features(features_matrix)
simple_features_matrix, norms

(array([[ 0.00680209,  0.00353021,  0.00583571],
        [ 0.00680209,  0.00768869,  0.00583571],
        [ 0.00680209,  0.00230361,  0.00389048],
        ..., 
        [ 0.00680209,  0.00305154,  0.00389048],
        [ 0.00680209,  0.00478673,  0.00583571],
        [ 0.00680209,  0.00305154,  0.00389048]]),
 array([  1.47013605e+02,   3.34257264e+05,   5.14075870e+02]))

In [13]:
weights = [1,4,1]

In [14]:
prediction = predict_outcome(simple_features_matrix, weights)

Compute the values of ro[i] for each feature in this simple model, using the formula given above, using the formula:
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
Hint: You can get a Numpy vector for feature_i using:
simple_feature_matrix[:,i]

In [15]:
ro_1 = np.dot(simple_features_matrix[:,1],(output_array - prediction + weights[1]*simple_features_matrix[:,1]))

In [16]:
ro_2 = np.dot(simple_features_matrix[:,2],(output_array - prediction + weights[2]*simple_features_matrix[:,2]))

10. Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

In [17]:
print ro_1, ro_2

87939470.8233 80966698.6662


In [18]:
range_l1penalty = [2*ro_2, 2*ro_1]

In [19]:
range_l1penalty

[161933397.33247885, 175878941.64650357]

Quiz Question: What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate?

Single Coordinate Descent Step

12. Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

e.g. in Python:

In [20]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_outcome(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot(feature_matrix[:,i],(output - prediction + weights[i]*feature_matrix[:,i]))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [21]:
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


#Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.


Return weights

The function should accept the following parameters:

Feature matrix
Output array
Initial weights
L1 penalty
Tolerance
e.g. in Python:



In [22]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    max_change = tolerance*2
    weights = initial_weights
    while max_change > tolerance:
        max_change = 0
        for i in range(len(weights)):
            old_weight_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change = np.abs(weights[i] - old_weight_i)
            if change>max_change:
                max_change = change
#         print max_change
    return weights

Using the following parameters, learn the weights on the sales dataset.

In [23]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

Quiz Question: What is the RSS of the learned model on the normalized dataset?



In [24]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix)

In [25]:
new_weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [26]:
print new_weights

[ 21624997.95951911  63157247.20788951         0.        ]


In [27]:
print np.sum((output-predict_outcome(normalized_simple_feature_matrix,new_weights))**2)

1.63049247672e+15


#Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets. If you are using GraphLab Create, call ‘random_split’ with .8 ratio and seed=0. Otherwise, please down the corresponding csv files from the downloads section.



Create a normalized feature matrix from the TRAINING data with the following set of features.

bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated
Make sure you store the norms for the normalization, since we’ll use them later.



In [28]:
train_features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated']

In [29]:
output = 'price'

In [30]:
train_feature_matrix, output_array = get_numpy_data(train, train_features, output)

In [31]:
train_normalized_features_matrix, normalization = normalize_features(train_feature_matrix)

First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights’ weights1e7’, you will need them later.

In [32]:
l1_penalty = 1e7
initialize_weights = np.zeros(14)
tolerance = 1

In [33]:
weights1e7 = lasso_cyclical_coordinate_descent(train_normalized_features_matrix, output_array,initialize_weights, l1_penalty, tolerance)

In [34]:
pd.Series(weights1e7,index=['intercept']+ train_features)

intercept        23864692.509538
bedrooms                0.000000
bathrooms               0.000000
sqft_living      30495548.132547
sqft_lot                0.000000
floors                  0.000000
waterfront        1901633.614756
view              5705765.016733
condition               0.000000
grade                   0.000000
sqft_above              0.000000
sqft_basement           0.000000
yr_built                0.000000
yr_renovated            0.000000
dtype: float64

In [35]:
l1_penalty = 1e8
initialize_weights = np.zeros(14)
tolerance = 1.0

In [36]:
weights1e8 = lasso_cyclical_coordinate_descent(train_normalized_features_matrix, output_array,initialize_weights, l1_penalty, tolerance)

In [37]:
pd.Series(weights1e8, index=['intercept']+train_features)

intercept        53621004.689715
bedrooms                0.000000
bathrooms               0.000000
sqft_living             0.000000
sqft_lot                0.000000
floors                  0.000000
waterfront              0.000000
view                    0.000000
condition               0.000000
grade                   0.000000
sqft_above              0.000000
sqft_basement           0.000000
yr_built                0.000000
yr_renovated            0.000000
dtype: float64

Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5. Call resulting weights ‘weights1e4’, you will need them later. (This case will take quite a bit longer to converge than the others above.)

In [38]:
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(train_normalized_features_matrix, output_array,initialize_weights, l1_penalty, tolerance)

In [39]:
pd.Series(weights1e4, index=['intercept']+train_features)

intercept        57481091.133021
bedrooms        -13652628.540224
bathrooms        12462713.071262
sqft_living      57942788.373312
sqft_lot         -1475769.694276
floors           -4904547.755466
waterfront        5349050.186362
view              5845253.562136
condition         -416038.969813
grade             2682274.594885
sqft_above         242649.685551
sqft_basement    -1285549.667681
yr_built        -54779474.227684
yr_renovated      2167703.066102
dtype: float64

#Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights. To use these weights on a test set, we must normalize the test data in the same way. Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:

In this case, we must scale the resulting weights so that we can make predictions with original features:

Store the norms of the original features to a vector called ‘norms’:


test_features_matrix, norms = normalize_features(train_feature_matrix)

Run Lasso on the normalized features and obtain a ‘weights’ vector
Compute the weights for the original features by performing element-wise division, i.e.


weights_normalized = weights / norms

Now, we can apply weights_normalized to the test data, without normalizing it!

In [41]:
weights1e7_normalized = weights1e7 / normalization
weights1e8_normalized = weights1e8 / normalization
weights1e4_normalized = weights1e4 / normalization
weights1e8

array([ 57481091.13302054, -13652628.5402242 ,  12462713.07126241,
        57942788.37331215,  -1475769.69427564,  -4904547.75546551,
         5349050.18636169,   5845253.56213634,   -416038.96981256,
         2682274.59488508,    242649.68555077,  -1285549.66768121,
       -54779474.22768354,   2167703.06610234])

In [41]:
(test_feature_matrix, test_output) = get_numpy_data(test, train_features, 'price')

In [42]:
print sum((test_output - predict_outcome(test_feature_matrix,weights1e4_normalized) )**2)

1.29085259902e+14


In [43]:
print sum((test_output - predict_outcome(test_feature_matrix,weights1e7_normalized) )**2)

1.63103564165e+14


In [44]:
print sum((test_output - predict_outcome(test_feature_matrix,weights1e8_normalized) )**2)

1.29085259902e+14
